In [1]:
import pandas as pd
import numpy as np
import copy, time, os
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
print(os.listdir('data/'))

['enron61702insiderpay.pdf', 'sample_submission.csv', 'test_features.csv', 'train_data.csv']


In [3]:
data_path = 'data/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')
df_sample_submission = pd.read_csv(data_path + 'sample_submission.csv')
df_train.shape, df_test.shape, df_sample_submission.shape

((113, 22), (33, 21), (33, 2))

In [4]:
df_train.head()

name      bonus  deferral_payments  deferred_income  \
0        RICE KENNETH D  1750000.0                NaN       -3504386.0   
1    SKILLING JEFFREY K  5600000.0                NaN              NaN   
2            SHELBY REX   200000.0                NaN          -4167.0   
3      KOPPER MICHAEL J   800000.0                NaN              NaN   
4  CALGER CHRISTOPHER F  1250000.0                NaN        -262500.0   

   director_fees                 email_address  exercised_stock_options  \
0            NaN            ken.rice@enron.com               19794175.0   
1            NaN       jeff.skilling@enron.com               19250000.0   
2            NaN          rex.shelby@enron.com                1624396.0   
3            NaN      michael.kopper@enron.com                      NaN   
4            NaN  christopher.calger@enron.com                      NaN   

   expenses  from_messages  from_poi_to_this_person  ...  long_term_incentive  \
0   46950.0           18.0                     42.0  ...            1617011.0   
1   29336.0          108.0                     88.0  ...            1920000.0   
2   22884.0           39.0                     13.0  ...                  NaN   
3  118134.0            NaN                      NaN  ...             602671.0   
4   35818.0          144.0                    199.0  ...             375304.0   

       other   poi  restricted_stock  restricted_stock_deferred     salary  \
0   174839.0  True         2748364.0                        NaN   420636.0   
1    22122.0  True         6843672.0                        NaN  1111258.0   
2  1573324.0  True          869220.0                        NaN   211844.0   
3   907502.0  True          985032.0                        NaN   224305.0   
4      486.0  True          126027.0                        NaN   240189.0   

   shared_receipt_with_poi  to_messages  total_payments  total_stock_value  
0                    864.0        905.0        505050.0         22542539.0  
1                   2042.0       3627.0       8682716.0         26093672.0  
2                     91.0        225.0       2003885.0          2493616.0  
3                      NaN          NaN       2652612.0           985032.0  
4                   2188.0       2598.0       1639297.0           126027.0  

[5 rows x 22 columns]

In [5]:
train_Y = pd.DataFrame(LabelEncoder().fit_transform(df_train['poi']))
df_train['poi'] = train_Y
df_train.pop('poi')
train_Y

0
0    1
1    1
2    1
3    1
4    1
..  ..
108  0
109  0
110  0
111  0
112  0

[113 rows x 1 columns]

In [6]:
#合併資料 一起做特徵工程
train_num = train_Y.shape[0]
all_data = pd.concat([df_train, df_test])
all_data.shape

(146, 21)

In [7]:
name = df_test['name']
all_data.pop('name')

0           RICE KENNETH D
1       SKILLING JEFFREY K
2               SHELBY REX
3         KOPPER MICHAEL J
4     CALGER CHRISTOPHER F
              ...         
28         BIBI PHILIPPE A
29         SHERRIFF JOHN R
30            GIBBS DANA R
31          LINDHOLM TOD A
32         MCMAHON JEFFREY
Name: name, Length: 146, dtype: object

In [8]:
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
na_check(all_data)

Missing Ratio
loan_advances                  97.260274
director_fees                  88.356164
restricted_stock_deferred      87.671233
deferral_payments              73.287671
deferred_income                66.438356
long_term_incentive            54.794521
bonus                          43.835616
from_messages                  41.095890
from_poi_to_this_person        41.095890
from_this_person_to_poi        41.095890

In [9]:
#all_data['email_address'] = pd.DataFrame(LabelEncoder().fit_transform(all_data['email_address'].fillna('NAN')))
all_data.pop('email_address')

0               ken.rice@enron.com
1          jeff.skilling@enron.com
2             rex.shelby@enron.com
3         michael.kopper@enron.com
4     christopher.calger@enron.com
                  ...             
28         philippe.bibi@enron.com
29         john.sherriff@enron.com
30            dana.gibbs@enron.com
31          tod.lindholm@enron.com
32       jeffrey.mcmahon@enron.com
Name: email_address, Length: 146, dtype: object

In [10]:
#MAIL特徵 ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']
#for c in ['to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']:
#    all_data[c] = all_data[c].fillna(0)

In [11]:
#all_data.pop('total_payments')
#all_data.pop('total_stock_value')

In [10]:
for c in ['loan_advances','director_fees','restricted_stock_deferred','deferral_payments','deferred_income','long_term_incentive','bonus']:
    all_data[c] = all_data[c].fillna(0)
#     all_data[c] = np.log1p(all_data[c]).fillna(0)
na_check(all_data)

Missing Ratio
to_messages                  41.095890
shared_receipt_with_poi      41.095890
from_this_person_to_poi      41.095890
from_poi_to_this_person      41.095890
from_messages                41.095890
other                        36.301370
salary                       34.931507
expenses                     34.931507
exercised_stock_options      30.136986
restricted_stock             24.657534

In [11]:
for c in ['to_messages','shared_receipt_with_poi','from_this_person_to_poi','from_poi_to_this_person','from_messages','other','salary','expenses','exercised_stock_options','restricted_stock','total_payments','total_stock_value']:
    all_data[c] = all_data[c].fillna(0)
#     all_data[c] = np.log1p(all_data[c]).fillna(0)
na_check(all_data)

Empty DataFrame
Columns: [Missing Ratio]
Index: []

In [25]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# corr = all_data.corr()
# sns.heatmap(corr)
# plt.show()

In [26]:
# corr

In [27]:
# high_list = list(corr[(corr['poi']>0.25) | (corr['poi']<-0.25)].index)
# high_list.pop(5)
# print(high_list)

In [12]:
MMEncoder = MinMaxScaler()
train_X = MMEncoder.fit_transform(all_data)
# estimator = GradientBoostingClassifier()
# cross_val_score(estimator, train_X, train_Y, cv=5).mean()

In [18]:
#財務特徵  ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees']


In [14]:
#all_data['bonus'] = all_data['bonus'].fillna(0)

# for c in ['total_payments', 'salary', 'deferral_payments', 'loan_advances', 'deferred_income',  'expenses', 'other', 'long_term_incentive', 'director_fees']:
#     all_data[c] = all_data[c].fillna(all_data[c].median())
#na_check(all_data)

In [15]:
#for c in ['total_stock_value','restricted_stock_deferred','exercised_stock_options','restricted_stock']:
#    all_data[c] = all_data[c].fillna(0)
#na_check(all_data)

In [12]:
# MMEncoder = MinMaxScaler()
# for col in all_data.columns:
#     all_data[col] = MMEncoder.fit_transform(all_data[col].values.reshape(-1,1))


In [13]:
all_data.describe()

bonus  deferral_payments  deferred_income  director_fees  \
count  1.460000e+02       1.460000e+02     1.460000e+02   1.460000e+02   
mean   1.333474e+06       4.387965e+05    -3.827622e+05   1.942249e+04   
std    8.094029e+06       2.741325e+06     2.378250e+06   1.190543e+05   
min    0.000000e+00      -1.025000e+05    -2.799289e+07   0.000000e+00   
25%    0.000000e+00       0.000000e+00    -3.792600e+04   0.000000e+00   
50%    3.000000e+05       0.000000e+00     0.000000e+00   0.000000e+00   
75%    8.000000e+05       9.684500e+03     0.000000e+00   0.000000e+00   
max    9.734362e+07       3.208340e+07     0.000000e+00   1.398517e+06   

       exercised_stock_options      expenses  from_messages  \
count             1.460000e+02  1.460000e+02     146.000000   
mean              4.182736e+06  7.074827e+04     358.602740   
std               2.607040e+07  4.327163e+05    1441.259868   
min               0.000000e+00  0.000000e+00       0.000000   
25%               0.000000e+00  0.000000e+00       0.000000   
50%               6.082935e+05  2.018200e+04      16.500000   
75%               1.714221e+06  5.374075e+04      51.250000   
max               3.117640e+08  5.235198e+06   14368.000000   

       from_poi_to_this_person  from_this_person_to_poi  loan_advances  \
count               146.000000               146.000000   1.460000e+02   
mean                 38.226027                24.287671   1.149658e+06   
std                  73.901124                79.278206   9.649342e+06   
min                   0.000000                 0.000000   0.000000e+00   
25%                   0.000000                 0.000000   0.000000e+00   
50%                   2.500000                 0.000000   0.000000e+00   
75%                  40.750000                13.750000   0.000000e+00   
max                 528.000000               609.000000   8.392500e+07   

       long_term_incentive         other  restricted_stock  \
count         1.460000e+02  1.460000e+02      1.460000e+02   
mean          6.646839e+05  5.854318e+05      1.749257e+06   
std           4.046072e+06  3.682345e+06      1.089995e+07   
min           0.000000e+00  0.000000e+00     -2.604490e+06   
25%           0.000000e+00  0.000000e+00      8.115000e+03   
50%           0.000000e+00  9.595000e+02      3.605280e+05   
75%           3.750648e+05  1.506065e+05      8.145280e+05   
max           4.852193e+07  4.266759e+07      1.303223e+08   

       restricted_stock_deferred        salary  shared_receipt_with_poi  \
count               1.460000e+02  1.460000e+02               146.000000   
mean                2.051637e+04  3.658114e+05               692.986301   
std                 1.439661e+06  2.203575e+06              1072.969492   
min                -7.576788e+06  0.000000e+00                 0.000000   
25%                 0.000000e+00  0.000000e+00                 0.000000   
50%                 0.000000e+00  2.105960e+05               102.500000   
75%                 0.000000e+00  2.708505e+05               893.500000   
max                 1.545629e+07  2.670423e+07              5521.000000   

        to_messages  total_payments  total_stock_value  
count    146.000000    1.460000e+02       1.460000e+02  
mean    1221.589041    4.350622e+06       5.846018e+06  
std     2226.770637    2.693448e+07       3.624681e+07  
min        0.000000    0.000000e+00      -4.409300e+04  
25%        0.000000    9.394475e+04       2.288695e+05  
50%      289.000000    9.413595e+05       9.659550e+05  
75%     1585.750000    1.968287e+06       2.319991e+06  
max    15149.000000    3.098866e+08       4.345095e+08

In [21]:
# ['salary', 'deferral_payments', 'loan_advances', 'bonus', 'deferred_income',  'expenses', 'other', 'long_term_incentive', 'director_fees']
#all_data['total_payments']



In [14]:
all_data_train = all_data[:train_num]
all_data_test = all_data[train_num:]
all_data_train.shape, all_data_test.shape , train_Y.shape

((113, 19), (33, 19), (113, 1))

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(all_data_train, train_Y, test_size=0.2, random_state=22)

In [17]:
#LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, Y_train)
lr_pred = lr.predict(X_test)
lr_pred_proba = lr.predict_proba(X_test)

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, lr_pred_proba[:,1]) 
print('AUC:',metrics.auc(fpr, tpr)) 

AUC: 0.3666666666666667


In [18]:
#RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=1)
rfc.fit(X_train, Y_train)
rfc_pred = rfc.predict(X_test)
rfc_pred_proba = rfc.predict_proba(X_test)

fpr, tpr, thresholds = metrics.roc_curve(Y_test, rfc_pred_proba[:,1]) 
print('AUC:',metrics.auc(fpr, tpr)) 

AUC: 0.8166666666666668


In [19]:
#GradientBoosting
gdbc = GradientBoostingClassifier(n_estimators=200, max_depth=5)
gdbc.fit(X_train, Y_train)
gdbc_pred = gdbc.predict(X_test)
gdbc_pred_proba = gdbc.predict_proba(X_test)

fpr, tpr, thresholds = metrics.roc_curve(Y_test, gdbc_pred_proba[:,1]) 
print('AUC:',metrics.auc(fpr, tpr)) 

AUC: 0.55


In [22]:
# blending_pred = lr_pred_proba[:,1]*0.05 + rfc_pred_proba[:,1]*0.8 + gdbc_pred_proba[:,1]*0.15
# fpr, tpr, thresholds = metrics.roc_curve(Y_test, blending_pred) 
# print('AUC:',metrics.auc(fpr, tpr)) 

AUC: 0.7999999999999999


In [20]:
#LogisticRegression
# lr_pred = lr.predict(all_data_test)
lr_pred_proba = lr.predict_proba(all_data_test)

#RandomForest
# rfc_pred = rfc.predict(all_data_test)
rfc_pred_proba = rfc.predict_proba(all_data_test)

#GradientBoosting
# gdbc_pred = gdbc.predict(all_data_test)
gdbc_pred_proba = gdbc.predict_proba(all_data_test)

blending_pred = lr_pred_proba[:,1]*0.05 + rfc_pred_proba[:,1]*0.8 + gdbc_pred_proba[:,1]*0.15

In [23]:
sub = pd.DataFrame({'name': name, 'poi': blending_pred})
sub.to_csv('Midterm_exam.csv', index=False)

In [42]:
from sklearn.model_selection import KFold, GridSearchCV
# 設定要訓練的超參數組合
n_estimators = [100, 150, 200, 250, 300]
max_depth = [1, 2, 3, 4, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(gdbc, param_grid, scoring="roc_auc", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, Y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.3s


Best Accuracy: 0.753125 using {'max_depth': 5, 'n_estimators': 200}


[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    2.0s finished


In [43]:
# 設定要訓練的超參數組合
n_estimators = [100, 150, 200, 250, 300]
max_depth = [1, 2, 3, 4, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(rfc, param_grid, scoring="roc_auc", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, Y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  94 out of 125 | elapsed:    2.0s remaining:    0.6s


Best Accuracy: 0.756250 using {'max_depth': 1, 'n_estimators': 200}


[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    2.6s finished


In [85]:
#df_sample_submission['poi'].map(lambda x:1 if x>0.5 else  0)